In [1]:
#import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
pd.set_option('display.max_columns', 200)
import re
import collections

In [2]:
#read in data from ?fao.csv? and display head for initial examination
csv_file = "../Resources/fao.csv"
fao_data_df = pd.read_csv(csv_file, encoding="ANSI")#encoding="latin-1" for mac.
fao_data_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [3]:
# group FAO data by 'Area' (country) and 'Item' (category) 

cntry_item_grp = fao_data_df.groupby(['Area', 'Item']).sum()
cntry_item_grp.head()

Area Code  Item Code  Element Code  latitude  \
Area        Item                                                                
Afghanistan Alcoholic Beverages          2       2924          5142     33.94   
            Animal fats                  2       2946          5142     33.94   
            Apples and products          2       2617          5142     33.94   
            Bananas                      2       2615          5142     33.94   
            Barley and products          4       5026         10663     67.88   

                                 longitude  Y1961  Y1962  Y1963  Y1964  Y1965  \
Area        Item                                                                
Afghanistan Alcoholic Beverages      67.71    0.0    0.0    0.0    0.0    0.0   
            Animal fats              67.71   20.0   20.0   21.0   22.0   24.0   
            Apples and products      67.71   14.0   14.0   14.0   17.0   19.0   
            Bananas                  67.71    0.0    0.0    0.0    0.0    0.0   
            Barley and products     135.42  313.0  313.0  313.0  314.0  314.0   

                                 Y1966  Y1967  Y1968  Y1969  Y1970  Y1971  \
Area        Item                                                            
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           25.0   27.0   29.0   29.0   27.0   26.0   
            Apples and products   22.0   24.0   25.0   26.0   24.0   20.0   
            Bananas                0.0    0.0    0.0    0.0    0.0    0.0   
            Barley and products  312.0  296.0  299.0  303.0  308.0  294.0   

                                 Y1972  Y1973  Y1974  Y1975  Y1976  Y1977  \
Area        Item                                                            
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           25.0   27.0   28.0   29.0   31.0   31.0   
            Apples and products   20.0   22.0   21.0   21.0   21.0   13.0   
            Bananas                0.0    0.0    0.0    0.0    0.0    0.0   
            Barley and products  289.0  297.0  316.0  321.0  335.0  245.0   

                                 Y1978  Y1979  Y1980  Y1981  Y1982  Y1983  \
Area        Item                                                            
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           31.0   31.0   31.0   32.0   32.0   32.0   
            Apples and products   18.0   19.0   17.0   18.0   16.0   15.0   
            Bananas                1.0    1.0    1.0    9.0   11.0    4.0   
            Barley and products  268.0  262.0  266.0  249.0  229.0  220.0   

                                 Y1984  Y1985  Y1986  Y1987  Y1988  Y1989  \
Area        Item                                                            
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           30.0   26.0   20.0   22.0   23.0   24.0   
            Apples and products   13.0   15.0   16.0   11.0   14.0   16.0   
            Bananas                3.0    4.0    5.0    9.0    6.0    0.0   
            Barley and products  211.0  199.0  191.0  191.0  195.0  191.0   

                                 Y1990  Y1991  Y1992  Y1993  Y1994  Y1995  \
Area        Item                                                            
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           29.0   30.0   30.0   33.0   37.0   50.0   
            Apples and products   16.0   15.0   15.0   15.0   14.0   12.0   
            Bananas                0.0    1.0    4.0    3.0    1.0    2.0   
            Barley and products  178.0  175.0  160.0  205.0  189.0  166.0   

                                 Y1996  Y1997  Y1998  Y1999  Y2000  Y2001  \
Area        Item                                                            
Afghanistan Alcoholic Beverages    0.0    0

In [4]:
#create a list from a unique set of the 'Item' categories from initial dataset
# create a dataframe to be exported to csv
FAO_ctgry_df = pd.DataFrame({'FAO_Category': list(set(fao_data_df['Item']))})
#FAO_ctgry_df.to_csv('FAO_ctgry_df.csv')
FAO_ctgry_df.head()

,FAO_Category
0,Fruits - Excluding Wine
1,Vegetable Oils
2,"Fats, Animals, Raw"
3,Poultry Meat
4,Peas


Analyze categories and create column of normalized category 'bins' in excel

In [5]:
#read category dataframe with new column set index to normalized category
csv_file = "./../FAO_norm_ctgry_df.csv"
FAO_nrm_ctgry_df = pd.read_csv(csv_file)
FAO_nrm_ctgry_df = FAO_nrm_ctgry_df.set_index('Normalized Category')
FAO_nrm_ctgry_df.head()
    

,FAO_Category
Normalized Category,
Oil,"Oilcrops, Other"
Grain,"Pulses, Other and products"
Spice,Pepper
Grain,Cereals - Excluding Beer
Plant,Cassava and products


In [6]:
#print statements to test output of itterows functions on naked row element (nothing in [])
#create list of tuples as key, value pairs to map a dictionary of categories
fao_cat_tuples= []
for index, row in FAO_nrm_ctgry_df.iterrows():
    #print(index)
    #print("--------------------------------")
    #print(row['FAO_Category'])
    #print("--------------------------------")
    fao_cat_tuples.append(tuple([index, row['FAO_Category']]))
fao_cat_tuples

[('Oil', 'Oilcrops, Other'),
 ('Grain', 'Pulses, Other and products'),
 ('Spice', 'Pepper'),
 ('Grain', 'Cereals - Excluding Beer'),
 ('Plant', 'Cassava and products'),
 ('Fruit', 'Dates'),
 ('Seafood', 'Cephalopods'),
 ('Oil', 'Maize Germ Oil'),
 ('Seed', 'Nuts and products'),
 ('Alcohol', 'Cereals, Other'),
 ('Fruit', 'Bananas'),
 ('Dry_Goods', 'Cocoa Beans and products'),
 ('Meat', 'Poultry Meat'),
 ('Meat', 'Mutton & Goat Meat'),
 ('Meat', 'Pigmeat'),
 ('Seed', 'Cottonseed'),
 ('Meat', 'Bovine Meat'),
 ('Dry_Goods', 'Stimulants'),
 ('Meat', 'Offals'),
 ('Fruit', 'Pineapples and products'),
 ('Oil', 'Fish, Body Oil'),
 ('Dairy', 'Cream'),
 ('Grain', 'Rice (Milled Equivalent)'),
 ('Seafood', 'Aquatic Products, Other'),
 ('Seafood', 'Crustaceans'),
 ('Meat', 'Offals, Edible'),
 ('Oil', 'Cottonseed Oil'),
 ('Coconut', 'Coconuts - Incl Copra'),
 ('Oil', "Ricebran Oil'"),
 ('Meat', 'Freshwater Fish'),
 ('Dairy', 'Butter, Ghee'),
 ('Meat', 'Meat, Other'),
 ('Fruit', 'Grapes and products (

In [7]:
#create dictionary mapping FAO_Categries to their normalized counterparts
# https://stackoverflow.com/questions/1747817/create-a-dictionary-with-list-comprehension-in-python

fao_cat_dict = {}
for (key, value) in fao_cat_tuples:
    if key not in fao_cat_dict:
        fao_cat_dict[key] = []
    fao_cat_dict[key].append(value)
fao_cat_dict

{'Oil': ['Oilcrops, Other',
  'Maize Germ Oil',
  'Fish, Body Oil',
  'Cottonseed Oil',
  "Ricebran Oil'",
  'Oilcrops',
  'Sunflowerseed Oil',
  'Fish, Liver Oil',
  'Rape and Mustard Oil',
  'Oilcrops Oil, Other',
  'Coconut Oil',
  'Olive Oil',
  'Sesameseed Oil',
  'Palmkernel Oil',
  'Vegetable Oils',
  'Palm Oil',
  'Soyabean Oil',
  "Oil', 'Groundnut Oil"],
 'Grain': ['Pulses, Other and products',
  'Cereals - Excluding Beer',
  'Rice (Milled Equivalent)',
  'Barley and products',
  'Oats',
  'Maize and products',
  'Beans',
  'Rye and products',
  'Pulses',
  'Millet and products',
  'Sorghum and products',
  'Wheat and products'],
 'Spice': ['Pepper', 'Spices', 'Spices, Other', 'Cloves'],
 'Plant': ['Cassava and products',
  'Starchy Roots',
  'Aquatic Plants',
  'Yams',
  'Plantains',
  'Sweet potatoes',
  'Onions',
  'Potatoes and products'],
 'Fruit': ['Dates',
  'Bananas',
  'Pineapples and products',
  'Grapes and products (excl wine)',
  'Lemons, Limes and products',
  '

In [8]:
fao_cat_dict = {}
for (key, value) in fao_cat_tuples:
    fao_cat_dict.setdefault(key, []).append(value)
fao_cat_dict

{'Oil': ['Oilcrops, Other',
  'Maize Germ Oil',
  'Fish, Body Oil',
  'Cottonseed Oil',
  "Ricebran Oil'",
  'Oilcrops',
  'Sunflowerseed Oil',
  'Fish, Liver Oil',
  'Rape and Mustard Oil',
  'Oilcrops Oil, Other',
  'Coconut Oil',
  'Olive Oil',
  'Sesameseed Oil',
  'Palmkernel Oil',
  'Vegetable Oils',
  'Palm Oil',
  'Soyabean Oil',
  "Oil', 'Groundnut Oil"],
 'Grain': ['Pulses, Other and products',
  'Cereals - Excluding Beer',
  'Rice (Milled Equivalent)',
  'Barley and products',
  'Oats',
  'Maize and products',
  'Beans',
  'Rye and products',
  'Pulses',
  'Millet and products',
  'Sorghum and products',
  'Wheat and products'],
 'Spice': ['Pepper', 'Spices', 'Spices, Other', 'Cloves'],
 'Plant': ['Cassava and products',
  'Starchy Roots',
  'Aquatic Plants',
  'Yams',
  'Plantains',
  'Sweet potatoes',
  'Onions',
  'Potatoes and products'],
 'Fruit': ['Dates',
  'Bananas',
  'Pineapples and products',
  'Grapes and products (excl wine)',
  'Lemons, Limes and products',
  '

In [9]:
# https://stackoverflow.com/questions/1747817/create-a-dictionary-with-list-comprehension-in-python

fao_cat_dict = collections.defaultdict(set) # ... dict(list) ... [k].append ...
#fao_cat_dict = {fao_cat_dict[key] for (key, value) in fao_cat_tuples}
for (k, v) in fao_cat_tuples:
    fao_cat_dict[k].add(v)
fao_cat_dict

defaultdict(set,
            {'Oil': {'Coconut Oil',
              'Cottonseed Oil',
              'Fish, Body Oil',
              'Fish, Liver Oil',
              'Maize Germ Oil',
              "Oil', 'Groundnut Oil",
              'Oilcrops',
              'Oilcrops Oil, Other',
              'Oilcrops, Other',
              'Olive Oil',
              'Palm Oil',
              'Palmkernel Oil',
              'Rape and Mustard Oil',
              "Ricebran Oil'",
              'Sesameseed Oil',
              'Soyabean Oil',
              'Sunflowerseed Oil',
              'Vegetable Oils'},
             'Grain': {'Barley and products',
              'Beans',
              'Cereals - Excluding Beer',
              'Maize and products',
              'Millet and products',
              'Oats',
              'Pulses',
              'Pulses, Other and products',
              'Rice (Milled Equivalent)',
              'Rye and products',
              'Sorghum and products',
            

In [10]:
# Import second database.

csv_file = "../Resources/en.openfoodfacts.org.products.tsv"
food_data_df = pd.read_csv(csv_file, sep='\t', low_memory=False)
food_data_df.head()


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, cor

In [11]:
#print formatted for easy read
for k,v in fao_cat_dict.items():
    print(k)
    print('--------------------------')
    print(v)
    print('--------------------------')

Oil
--------------------------
{"Ricebran Oil'", 'Vegetable Oils', 'Fish, Liver Oil', 'Olive Oil', 'Palm Oil', 'Cottonseed Oil', 'Rape and Mustard Oil', 'Oilcrops', 'Fish, Body Oil', 'Sunflowerseed Oil', 'Coconut Oil', 'Sesameseed Oil', 'Oilcrops Oil, Other', "Oil', 'Groundnut Oil", 'Palmkernel Oil', 'Maize Germ Oil', 'Soyabean Oil', 'Oilcrops, Other'}
--------------------------
Grain
--------------------------
{'Beans', 'Rice (Milled Equivalent)', 'Pulses, Other and products', 'Cereals - Excluding Beer', 'Pulses', 'Rye and products', 'Millet and products', 'Wheat and products', 'Sorghum and products', 'Barley and products', 'Maize and products', 'Oats'}
--------------------------
Spice
--------------------------
{'Pepper', 'Cloves', 'Spices, Other', 'Spices'}
--------------------------
Plant
--------------------------
{'Cassava and products', 'Aquatic Plants', 'Starchy Roots', 'Sweet potatoes', 'Plantains', 'Potatoes and products', 'Yams', 'Onions'}
--------------------------
Fruit
--

In [12]:
#print formatted for easy copy/paste
for k,v in fao_cat_dict.items():
    print(f'{k} = {v}')

Oil = {"Ricebran Oil'", 'Vegetable Oils', 'Fish, Liver Oil', 'Olive Oil', 'Palm Oil', 'Cottonseed Oil', 'Rape and Mustard Oil', 'Oilcrops', 'Fish, Body Oil', 'Sunflowerseed Oil', 'Coconut Oil', 'Sesameseed Oil', 'Oilcrops Oil, Other', "Oil', 'Groundnut Oil", 'Palmkernel Oil', 'Maize Germ Oil', 'Soyabean Oil', 'Oilcrops, Other'}
Grain = {'Beans', 'Rice (Milled Equivalent)', 'Pulses, Other and products', 'Cereals - Excluding Beer', 'Pulses', 'Rye and products', 'Millet and products', 'Wheat and products', 'Sorghum and products', 'Barley and products', 'Maize and products', 'Oats'}
Spice = {'Pepper', 'Cloves', 'Spices, Other', 'Spices'}
Plant = {'Cassava and products', 'Aquatic Plants', 'Starchy Roots', 'Sweet potatoes', 'Plantains', 'Potatoes and products', 'Yams', 'Onions'}
Fruit = {'Fruits - Excluding Wine', 'Grapefruit and products', 'Grapes and products (excl wine)', 'Apples and products', 'Dates', 'Bananas', 'Citrus, Other', 'Lemons, Limes and products', 'Olives (including preserved

In [13]:
# OFD with ingredients for potential categorizations ordered by column with fewest NaN
food_ing_df = food_data_df[food_data_df.columns.difference(['states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]
food_ing_df = food_ing_df[['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
#food_ing_df = food_df.dropna(axis=0, subset=['categories_en'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['energy_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['proteins_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['carbohydrates_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['saturated-fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fiber_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['sugars_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['ingredients_text'])
food_ing_df.columns

Index(['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime',
       'code', 'countries_en', 'countries_tags', 'countries', 'product_name',
       'brands', 'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 

In [14]:
# create dataframe from ingredients list and pare down to main ingridient and remove non-english characters using regex
OFD_ing_df = pd.DataFrame({'OFD_Ingredients': list(set(food_ing_df['ingredients_text']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ing_df['Main_Ingredient'] = OFD_ing_df['OFD_Ingredients'].str.split(',').str[0]
OFD_ing_df = OFD_ing_df[~OFD_ing_df['Main_Ingredient'].astype(str).str.contains(r'[^A-Za-z]', regex=True)]
#OFD_ing_df
OFD_ing_df['Main_Ingredient'].to_csv('./../OFD_ing_df.csv')
OFD_Ing_List = list(OFD_ing_df['Main_Ingredient'])

In [15]:
len(OFD_Ing_List)

37355

In [16]:
# remove the columns with multirow text as initial cleanup to the massive open food database
mini_food_df = food_data_df[food_data_df.columns.difference(['ingredients_text', 'states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]


In [17]:
# define function to determine percentage of missing values in each column
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [18]:
#  run reduced dataframe through function to ascertain which columns contain lots of empty values
# sort values in ascending order so cleanest columns are at the top
miss_df = missing_values_table(mini_food_df)
miss_df = miss_df.sort_values('% of Total Values', ascending=True)
miss_df.head() 

Your selected dataframe has 156 columns.
There are 154 columns that have missing values.


,Missing Values,% of Total Values
created_t,3,0.0
url,26,0.0
creator,3,0.0
created_datetime,10,0.0
code,26,0.0


In [19]:
# create new dataframe with only desired columns in desired order with the good old copy-past

food_df = food_data_df[['created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
food_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
0,1474103866,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2016-09-17T09:17:46Z,0000000003087,France,en:france,en:FR,Farine de blé noir,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004530,United States,en:united-states,US,Banana Chips Sweetened (Whole),NaN,NaN,2243.0,3.57,0.00000,0.000,0.0,0.0,28.57,14.29,64.29,28.57,14.0,d,14.0,3.6,28 g (1 ONZ),NaN,0.018,0.0,0.000,0.0214,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004559,United States,en:united-states,US,Peanuts,Torn & Glasser,torn-glasser,1941.0,17.86,0.63500,0.250,0.0,0.0,17.86,17.86,60.71,0.00,0.0,b,0.0,7.1,28 g (0.25 cup),NaN,0.000,0.0,0.071,0.0000,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1489055731,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:35:31Z,0000000016087,United States,en:united-states,US,Organic Salted Nut Mix,Grizzlies,grizzlies,2540.0,17.86,1.22428,0.482,0.0,0.0,57.14,3.57,17.86,5.36,12.0,d,12.0,7.1,28 g (0.25 cup),NaN,NaN,NaN,0.143,NaN,0.00514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1489055653,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:34:13Z,0000000016094,United States,en:united-states,US,Organic Polenta,Bob's Red Mill,bob-s-red-mill,1552.0,8.57,NaN,NaN,0.0,0.0,1.43,NaN,77.14,NaN,NaN,NaN,NaN,5.7,35 g (0.25 cup),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# remove NaN values from all relevant columns
relevant_cats= ['categories_en', 'energy_100g', 'proteins_100g', 'carbohydrates_100g', 
                'fat_100g', 'saturated-fat_100g', 'fiber_100g', 'sugars_100g']
clean_df = food_df.dropna(axis=0, subset=['categories_en'])
clean_df = clean_df.dropna(axis=0, subset=['energy_100g'])
clean_df = clean_df.dropna(axis=0, subset=['proteins_100g'])
clean_df = clean_df.dropna(axis=0, subset=['carbohydrates_100g'])
clean_df = clean_df.dropna(axis=0, subset=['saturated-fat_100g'])
clean_df = clean_df.dropna(axis=0, subset=['fiber_100g'])
clean_df = clean_df.dropna(axis=0, subset=['fat_100g'])
clean_df = clean_df.dropna(axis=0, subset=['sugars_100g'])
clean_df



,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
176,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.000,2.160000,0.850000,0.0,0.0,12.00,0.000,23.00,7.000,6.0,c,6.0,2.000,100 g,NaN,0.05600,NaN,0.0000,0.000000,0.000000,0.000000,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN
182,1484501528,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15T17:32:08Z,0000001938067,Canada,en:canada,Canada,Chaussons tressés aux pommes,Kirkland Signature,kirkland-signature,1090.0,3.330,0.647000,0.255000,0.0,0.0,10.70,24.700,38.70,2.000,9.0,c,9.0,2.000,150 g / 1 chausson,"en:e202,en:e211,en:e330,en:e509,en:e920",0.00533,0.667,0.0133,0.001600,0.000480,0.000000,Biscuits and cakes,Sugary snacks,1.200 kg,"en:sugary-snacks,en:biscuits-and-cakes,en:past...",Sugary snacks,en:sugary-snacks,"Snacks sucrés,Biscuits et gâteaux,Pâtisseries","Sugary snacks,Biscuits and cakes,Pastries",frais,Frais,NaN,NaN,Brossard Québec,NaN,NaN,NaN,Pâtisserie,Costco,Brossard Québec,brossard-quebec,NaN
183,1488464896,http://world-en.openfoodfacts.org/product/0000...,b7,2017-03-02T14:28:16Z,0000004302544,Canada,en:canada,Canada,Pain Burger Artisan,Kirkland Signature,kirkland-signature,1160.0,10.000,1.520000,0.600000,0.0,0.0,1.11,1.110,53.30,0.333,1.0,b,1.0,2.220,90 g / 1 pain,NaN,0.00000,0.000,0.0222,0.000000,0.006000,0.000000,unknown,unknown,1.008 kg / 12 pain,fr:boulange,fr:Boulange,fr:boulange,Boulange,fr:Boulange,"frais,plastique","Frais,plastique",NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Pain,Costco,"Brossard,Québec","brossard,quebec",NaN
185,1420150193,http://world-en.openfoodfacts.org/product/0000...,teolemon,2015-01-01T22:09:53Z,0000007020254,France,en:france,France,Root Beer,A&W,a-w,215.0,0.000,0.061600,0.024200,0.0,0.0,0.00,13.600,14.20,0.000,18.0,e,3.0,0.000,33 cl,"en:e150,en:e211,en:e999",NaN,NaN,NaN,NaN,NaN,NaN,Sweetened beverages,Beverages,33 cl e,"en:beverages,en:carbonated-drinks,en:sodas,en:...",Beverages,en:beverages,"Boissons,Boissons gazeuses,Sodas,Boissons sucr...","Beverages,Carbonated drinks,Sodas,Sugared beve...","canette,metal","Canette,Métal",http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,France,"Kosher,fr:Contient-des-ogm","Kascher,Contient des OGM","en:kosher,fr:contient-des-ogm",NaN,NaN,États-Unis,etats-unis,NaN
187,1491943307,http://world-en.openfoodfacts.org/product/0000...,b7,2017-04-11T20:41:47Z,0000008237798,Canada,en:canada,Canada,Quiche Lorraine,Kirkland Signature,kirkland-signature,478.0,5.360,0.499000,0.196000,0.0,0.0,6.79,0.714,7.86,2.860,2.0,b,2.0,0.357,280 g / 1/5 de la Quiche,"en:e235,en:e316,en:e407",0.01070,0.000,0.0286,0.000429,0.000643,0.000011,Pizza pies and quiche,Composite foods,1 400 kg,"en:meals,en:pizzas-pies-and-quiches,en:quiches...",Meals,en:meals,Quiches lorraines,"Meals,Pizzas pies and quiches,Quiches,fr:Quich...",frai,Frai,NaN,NaN,"Brossard,Québec",NaN,NaN

In [21]:
#create category dataframe from relevant clean_df and append a boolean column stating if category is in english
# 3,640 categoryies in original df, 1,287 after removing non-english; 1463 categories in clean_df, 483 after non-english filter
OFD_ctgry_df = pd.DataFrame({'OFD_Categories': list(set(clean_df['main_category']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].str.match(r'en:')
OFD_ctgry_df

,OFD_Categories,en_boolean
0,fr:mayonnaise-a-l-huile-d-olive,False
1,en:nans,True
2,en:goat-milk-butter,True
3,de:fruchtnektare,False
4,fr:boisson-au-cranberry-multifruits,False
5,fr:faisselles,False
6,en:milk-substitute,True
7,en:chicken-thighs,True
8,fr:fromage-a-pate-molle,False
9,en:formulated-supplementary-food,True


In [22]:
# reduce dataframe to only english (supposedly, lol) categories and strip the prefix
# make list from column
OFD_en_ctgry_df = OFD_ctgry_df.loc[OFD_ctgry_df['en_boolean'] == True, :].reset_index().drop(columns=['index', 'en_boolean'])
OFD_en_ctgry_df['OFD_Categories'] = OFD_en_ctgry_df['OFD_Categories'].str.slice(3)
OFD_en_ctgry_df.to_csv('./../OFD_en_ctgry_df.csv')

OFD_Categories = list(OFD_en_ctgry_df['OFD_Categories'])
OFD_Categories


['nans',
 'goat-milk-butter',
 'milk-substitute',
 'chicken-thighs',
 'formulated-supplementary-food',
 'slaty-snack',
 'lemongrass-paste',
 'chocolate-muffin',
 'farming-products',
 'cherry-juice',
 'cookies-chocolate',
 'pancakes-and-waffles',
 'kauwgom',
 'seafood',
 'aliments-d-origine-vegetale',
 'confectioner',
 'baked-beans-in-tomato-sauce',
 'corn-kernels',
 'soup-mix',
 'raw',
 'cheddar-baguettes',
 'feta-cheese',
 'saucisses-aux-lentilles',
 'noix-de-paprika',
 'lingonberry-jams',
 'easter-buns',
 'fats',
 'white-plain-flour',
 'steak-pie',
 'chilli-sauce',
 'tea-drinks',
 'dietary-supplements',
 'soupes',
 'crema-spalmabile',
 'mix',
 'bouillonterning',
 'yoghurts-op-een-laag-fruit',
 'non-dairy-soymilk',
 'hawaiian-sweet-rolls',
 'tiefkuhlpizza',
 'cranberry-juice-drink',
 'preparation-for-dessert',
 'vegetarian-chicken-fillets',
 'breaded-ham',
 'porridge-oats',
 'beef-jerky',
 'prepared-salads',
 'cheese-analogues',
 'stuffing',
 'hot-fudge',
 'whole-wheat-penne-rigate',


In [23]:
#read category dataframe with new column set index to normalized category
csv_file = "./../OFD_en_norm_ctgry_df.csv"
OFD_nrm_ctgry_df = pd.read_csv(csv_file)
OFD_nrm_ctgry_df = OFD_nrm_ctgry_df.set_index('normalized_category')
OFD_nrm_ctgry_df.head()

#print statements to test output of itterows functions on naked row element (nothing in [])
#create list of tuples as key, value pairs to map a dictionary of categories
ofd_cat_tuples= []
for index, row in OFD_nrm_ctgry_df.iterrows():
    #print(index)
    #print("--------------------------------")
    #print(row['OFD_Category'])
    #print("--------------------------------")
    ofd_cat_tuples.append(tuple([index, row['OFD_Category']]))
ofd_cat_tuples
ofd_cat_dict = {}
for (key, value) in ofd_cat_tuples:
    ofd_cat_dict.setdefault(key, []).append(value)
ofd_cat_dict

FileNotFoundError: File b'./../OFD_en_norm_ctgry_df.csv' does not exist

In [24]:
# compare original (no reset index) df to compare the two category lists which will become primary keys to link the databases
csv_file = "./../OFD_en_norm_ctgry_df.csv"
OFD_nrm_ctgry_df = pd.read_csv(csv_file)
ofd_nrm_ctgry = list(OFD_nrm_ctgry_df['normalized_category'])
set(ofd_nrm_ctgry)

FileNotFoundError: File b'./../OFD_en_norm_ctgry_df.csv' does not exist

In [25]:
# compare original (no reset index) df to compare the two category lists which will become primary keys to links the databases
csv_file = "FAO_norm_ctgry_df.csv"
FAO_nrm_ctgry_df = pd.read_csv(csv_file)
fao_nrm_ctgry = list(FAO_nrm_ctgry_df['Normalized Category'])
set(fao_nrm_ctgry)

{'Alcohol',
 'Coconut',
 'Dairy',
 'Dry_Goods',
 'Fruit',
 'Grain',
 'Infant',
 'Meat',
 'Oil',
 'Other',
 'Palm',
 'Plant',
 'Seafood',
 'Seed',
 'Soy',
 'Spice',
 'Sweet',
 'Vegetable'}

In [26]:
# https://stackoverflow.com/questions/6181935/how-do-you-create-different-variable-names-while-in-a-loop
# https://developers.google.com/edu/python/dict-files
#for k, v in cat_dict.items():
    #globals()['k%s' % k] = v
    #%(k) = %(v)


In [27]:
#for k in cat_dict.keys():
    #globals()['k%s' % k] = 'me'

In [28]:
for k, v in df.items():
    if 'one' in v:
        print('one')

NameError: name 'df' is not defined

In [29]:
# flatten list of list of strings into new list of strings

Oil = (('Oilcrops'), ('Oil'))
Grain = (('Pulse'), ('Cereal'), ('Rice'), ('Barley'), ('Oat'), ('Maize'), ('Bean'), ('Rye'), ('Millet'), ('Sorghum'), ('Wheat'), ('Lentil'), ('Lentil'))
Spice = (('Pepper'), ('Cloves'), ('Spice'))
Plant = (('Cassava'), ('Root'), ('Plant'), ('Yam'), ('Plantain'),  ('potatoes'), ('potato'), ('Onions'))
Fruit = (('Date'), ('Banana'), ('Pineapple'), ('Grape'), ('Lemon'), ('Lime'), ('Fruits'), ('Apples'), ('Olives'), ('Fruit'), ('Grapefruit'), ('Orange'), ('Mandarin'), ('Citrus'))
Seafood = (('Cephalopod'), ('Squid'), ('Cuttlefish'), ('Octopus'), ('Aquatic'), ('Crustacean'), ('Crab'), ('Shrimp'), ('Langoustine'), ('Lobster'), ('Sea'), ('Fish'), ('Pelagic'), ('Molluscs'), ('Clam'), ('Oyster'), ('Mussel'), ('Marine'), ('Seafood'), ('Shellfish'), ('Freshwater'))
Seed_Nut= (('Seed'), ('Nut'), ('Cottonseed'), ('Rapeseed'), ('Mustardseed'), ('Sunflower'), ('Sesame'), ('Groundnut'), ('Peanut'), ('Treenut'), ('Almond'), ('Walnut'), ('Pecan'), ('Cashew'), ('Macadamia'), ('Brazil'))
Alcohol = (('Alcohol'), ('Alcoholic'), ('Beer'), ('Wine'), ('Vodka'), ('Whiskey'), ('Rum'), ('Cider'))
Dry_Goods = (('Cocoa'), ('Coffee'), ('Tea'), ('Mate'))
Meat = (('Poultry'), ('Chicken'), ('Turkey'), ('Mutton'), ('Goat'), ('Lamb'), ('Pigmeat'), ('Pig'), ('Pork'), ('Bovine'), ('Cow'), ('Beef'), ('Offal'), ('Meat'), ('Egg'), ('Animal'), ('Fat'))
Dairy = (('Cream'), ('Butter'), ('Ghee'), ('Milk'), ('Butter'))
Sweet = (('Cane'), ('Honey'), ('Sugar'), ('Sweet'), ('Sweeteners'), ('Candy'), ('Choco'))
Infant = (('Infant food'))
Other = (('Miscellaneous'))
Vegetable = (('Pimento'), ('Vegetables'), ('Tomatoes'), ('Pea'), ('Vegetable'), ('Cabbage'), ('Broccoli'), ('Cauliflower'), ('Kale'), ('Sprout'), ('Cruciferous'), ('Cucumber'))
Palm = (('Palm'))
Soy = (('Soy'))
Coconut = (('Coconut'), ('Copra'))


list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']


map_list = [Oil, Grain, Spice, Plant, Fruit, Seafood, Seed_Nut, Alcohol, Dry_Goods, Meat, Dairy, Sweet, Infant, Other, Vegetable, Palm, Soy, Coconut]

cat_zip = list(zip(list_cats, map_list))

map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict

{'Oil': [('Oilcrops', 'Oil')],
 'Grain': [('Pulse',
   'Cereal',
   'Rice',
   'Barley',
   'Oat',
   'Maize',
   'Bean',
   'Rye',
   'Millet',
   'Sorghum',
   'Wheat',
   'Lentil',
   'Lentil')],
 'Spice': [('Pepper', 'Cloves', 'Spice')],
 'Plant': [('Cassava',
   'Root',
   'Plant',
   'Yam',
   'Plantain',
   'potatoes',
   'potato',
   'Onions')],
 'Fruit': [('Date',
   'Banana',
   'Pineapple',
   'Grape',
   'Lemon',
   'Lime',
   'Fruits',
   'Apples',
   'Olives',
   'Fruit',
   'Grapefruit',
   'Orange',
   'Mandarin',
   'Citrus')],
 'Seafood': [('Cephalopod',
   'Squid',
   'Cuttlefish',
   'Octopus',
   'Aquatic',
   'Crustacean',
   'Crab',
   'Shrimp',
   'Langoustine',
   'Lobster',
   'Sea',
   'Fish',
   'Pelagic',
   'Molluscs',
   'Clam',
   'Oyster',
   'Mussel',
   'Marine',
   'Seafood',
   'Shellfish',
   'Freshwater')],
 'Seed_Nut': [('Seed',
   'Nut',
   'Cottonseed',
   'Rapeseed',
   'Mustardseed',
   'Sunflower',
   'Sesame',
   'Groundnut',
   'Peanut',
 

In [30]:
# flatten list of list of strings into new list of strings
Oil = ['Oilcrops', 'Oil']
Grain = ['Pulse', 'Cereal', 'Rice', 'Barley', 'Oat', 'Maize', 'Bean', 'Rye', 'Millet', 'Sorghum', 'Wheat', 'Lentil', 'Lentil']
Spice = ['Pepper', 'Cloves', 'Spice']
Plant = ['Cassava', 'Root', 'Plant', 'Yam', 'Plantain',  'potatoes', 'potato', 'Onions']
Fruit = ['Date', 'Banana', 'Pineapple', 'Grape', 'Lemon', 'Lime', 'Fruits', 'Apples', 'Olives', 'Fruit', 'Grapefruit', 'Orange', 'Mandarin', 'Citrus']
Seafood = ['Cephalopod', 'Squid', 'Cuttlefish', 'Octopus', 'Aquatic', 'Crustacean', 'Crab', 'Shrimp', 'Langoustine', 'Lobster', 'Sea', 'Fish', 'Pelagic', 'Molluscs', 'Clam', 'Oyster', 'Mussel', 'Marine', 'Seafood', 'Shellfish', 'Freshwater']
Seed_Nut= ['Seed', 'Nut', 'Cottonseed', 'Rapeseed', 'Mustardseed', 'Sunflower', 'Sesame', 'Groundnut', 'Peanut', 'Treenut', 'Almond', 'Walnut', 'Pecan', 'Cashew', 'Macadamia', 'Brazil']
Alcohol = ['Alcohol', 'Alcoholic', 'Beer', 'Wine', 'Vodka', 'Whiskey', 'Rum', 'Cider']
Dry_Goods = ['Cocoa', 'Coffee', 'Tea', 'Mate']
Meat = ['Poultry', 'Chicken', 'Turkey', 'Mutton', 'Goat', 'Lamb', 'Pigmeat', 'Pig', 'Pork', 'Bovine', 'Cow', 'Beef', 'Offal', 'Meat', 'Egg', 'Animal', 'Fat']
Dairy = ['Cream', 'Butter', 'Ghee', 'Milk', 'Butter']
Sweet = ['Cane', 'Honey', 'Sugar', 'Sweet', 'Sweeteners', 'Candy', 'Choco']
Infant = ['Infant food']
Other = ['Miscellaneous']
Vegetable = ['Pimento', 'Vegetables', 'Tomatoes', 'Pea', 'Vegetable', 'Cabbage', 'Broccoli', 'Cauliflower', 'Kale', 'Sprout', 'Cruciferous', 'Cucumber']
Palm = ['Palm']
Soy = ['Soy']
Coconut = ['Coconut', 'Copra']
list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']

map_list = [Oil, Grain, Spice, Plant, Fruit, Seafood, Seed_Nut, Alcohol, Dry_Goods, Meat, Dairy, Sweet, Infant, Other, Vegetable, Palm, Soy, Coconut]

flat_list = []
for item in map_list:
    item = re.sub(r"'", '', str(item)).strip('[]')
    #item = ''.join(('\'', item, '\''))
    #item = item.strip('[]')
    #item = tuple(item)
    #str(item)
    #print(item)
    flat_list.append(item)

cat_zip = list(zip(list_cats, flat_list))

map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict

{'Oil': ['Oilcrops, Oil'],
 'Grain': ['Pulse, Cereal, Rice, Barley, Oat, Maize, Bean, Rye, Millet, Sorghum, Wheat, Lentil, Lentil'],
 'Spice': ['Pepper, Cloves, Spice'],
 'Plant': ['Cassava, Root, Plant, Yam, Plantain, potatoes, potato, Onions'],
 'Fruit': ['Date, Banana, Pineapple, Grape, Lemon, Lime, Fruits, Apples, Olives, Fruit, Grapefruit, Orange, Mandarin, Citrus'],
 'Seafood': ['Cephalopod, Squid, Cuttlefish, Octopus, Aquatic, Crustacean, Crab, Shrimp, Langoustine, Lobster, Sea, Fish, Pelagic, Molluscs, Clam, Oyster, Mussel, Marine, Seafood, Shellfish, Freshwater'],
 'Seed_Nut': ['Seed, Nut, Cottonseed, Rapeseed, Mustardseed, Sunflower, Sesame, Groundnut, Peanut, Treenut, Almond, Walnut, Pecan, Cashew, Macadamia, Brazil'],
 'Alcohol': ['Alcohol, Alcoholic, Beer, Wine, Vodka, Whiskey, Rum, Cider'],
 'Dry_Goods': ['Cocoa, Coffee, Tea, Mate'],
 'Meat': ['Poultry, Chicken, Turkey, Mutton, Goat, Lamb, Pigmeat, Pig, Pork, Bovine, Cow, Beef, Offal, Meat, Egg, Animal, Fat'],
 'Dairy': [

In [31]:
# dictionary from manually formatted list of strings feeds into list(zip()) correctly

Oil_String = 'Oilcrops, Oil'
Grain_String = 'Pulse, Cereal, Rice, Barley, Oat, Maize, Bean, Rye, Millet, Sorghum, Wheat, Lentil, Lentil' 
Spice_String = 'Pepper, Cloves, Spice'
Plant_String = 'Cassava, Root, Plant, Yam, Plantain,  potatoes, potato, Onions'
Fruit_String = 'Date, Banana, Pineapple, Grape, Lemon, Lime, Fruits, Apples, Olives, Fruit, Grapefruit, Orange, Mandarin, Citrus' 
Seafood_String = 'Cephalopod, Squid, Cuttlefish, Octopus, Aquatic, Crustacean, Crab, Shrimp, Langoustine, Lobster, Sea, Fish, Pelagic, Molluscs, Clam, Oyster, Mussel, Marine, Seafood, Shellfish, Freshwater'
Seed_Nut_String = 'Seed, Nut, Cottonseed, Rapeseed, Mustardseed, Sunflower, Sesame, Groundnut, Peanut, Treenut, Almond, Walnut, Pecan, Cashew, Macadamia, Brazil' 
Alcohol_String = 'Alcohol, Alcoholic, Beer, Wine, Vodka, Whiskey, Rum, Cider'
Dry_Goods_String = 'Cocoa, Coffee, Tea, Mate'
Meat_String = 'Poultry, Chicken, Turkey, Mutton, Goat, Lamb, Pigmeat, Pig, Pork, Bovine, Cow, Beef, Offal, Meat, Egg, Animal, Fat' 
Dairy_String = 'Cream, Butter, Ghee, Milk, Butter'
Sweet_String = 'Cane, Honey, Sugar, Sweet, Sweeteners, Candy, Choco'
Infant_String = 'Infant food'
Other_String = 'Miscellaneous'
Vegetable_String = 'Pimento, Vegetables, Tomatoes, Pea, Vegetable, Cabbage, Broccoli, Cauliflower, Kale, Sprout, Cruciferous, Cucumber'
Palm_String = 'Palm'
Soy_String = 'Soy'
Coconut_String = 'Coconut, Copra'

string_list = [Oil_String, Grain_String, Spice_String, Plant_String, Fruit_String, Seafood_String, Seed_Nut_String, Alcohol_String, Dry_Goods_String, Meat_String, Dairy_String, Sweet_String, Infant_String, Other_String, Vegetable_String, Palm_String, Soy_String, Coconut]

list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']

cat_zip = list(zip(list_cats, string_list))
cat_zip
map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict


{'Oil': ['Oilcrops, Oil'],
 'Grain': ['Pulse, Cereal, Rice, Barley, Oat, Maize, Bean, Rye, Millet, Sorghum, Wheat, Lentil, Lentil'],
 'Spice': ['Pepper, Cloves, Spice'],
 'Plant': ['Cassava, Root, Plant, Yam, Plantain,  potatoes, potato, Onions'],
 'Fruit': ['Date, Banana, Pineapple, Grape, Lemon, Lime, Fruits, Apples, Olives, Fruit, Grapefruit, Orange, Mandarin, Citrus'],
 'Seafood': ['Cephalopod, Squid, Cuttlefish, Octopus, Aquatic, Crustacean, Crab, Shrimp, Langoustine, Lobster, Sea, Fish, Pelagic, Molluscs, Clam, Oyster, Mussel, Marine, Seafood, Shellfish, Freshwater'],
 'Seed_Nut': ['Seed, Nut, Cottonseed, Rapeseed, Mustardseed, Sunflower, Sesame, Groundnut, Peanut, Treenut, Almond, Walnut, Pecan, Cashew, Macadamia, Brazil'],
 'Alcohol': ['Alcohol, Alcoholic, Beer, Wine, Vodka, Whiskey, Rum, Cider'],
 'Dry_Goods': ['Cocoa, Coffee, Tea, Mate'],
 'Meat': ['Poultry, Chicken, Turkey, Mutton, Goat, Lamb, Pigmeat, Pig, Pork, Bovine, Cow, Beef, Offal, Meat, Egg, Animal, Fat'],
 'Dairy': 

In [32]:
# dictionary from list of lists adds an unwanted layer to dictionary

Oil = ['Oilcrops', 'Oil']
Grain = ['Pulse', 'Cereal', 'Rice', 'Barley', 'Oat', 'Maize', 'Bean', 'Rye', 'Millet', 'Sorghum', 'Wheat', 'Lentil', 'Lentil']
Spice = ['Pepper', 'Cloves', 'Spice']
Plant = ['Cassava', 'Root', 'Plant', 'Yam', 'Plantain',  'potatoes', 'potato', 'Onions']
Fruit = ['Date', 'Banana', 'Pineapple', 'Grape', 'Lemon', 'Lime', 'Fruits', 'Apples', 'Olives', 'Fruit', 'Grapefruit', 'Orange', 'Mandarin', 'Citrus']
Seafood = ['Cephalopod', 'Squid', 'Cuttlefish', 'Octopus', 'Aquatic', 'Crustacean', 'Crab', 'Shrimp', 'Langoustine', 'Lobster', 'Sea', 'Fish', 'Pelagic', 'Molluscs', 'Clam', 'Oyster', 'Mussel', 'Marine', 'Seafood', 'Shellfish', 'Freshwater']
Seed_Nut= ['Seed', 'Nut', 'Cottonseed', 'Rapeseed', 'Mustardseed', 'Sunflower', 'Sesame', 'Groundnut', 'Peanut', 'Treenut', 'Almond', 'Walnut', 'Pecan', 'Cashew', 'Macadamia', 'Brazil']
Alcohol = ['Alcohol', 'Alcoholic', 'Beer', 'Wine', 'Vodka', 'Whiskey', 'Rum', 'Cider']
Dry_Goods = ['Cocoa', 'Coffee', 'Tea', 'Mate']
Meat = ['Poultry', 'Chicken', 'Turkey', 'Mutton', 'Goat', 'Lamb', 'Pigmeat', 'Pig', 'Pork', 'Bovine', 'Cow', 'Beef', 'Offal', 'Meat', 'Egg', 'Animal', 'Fat']
Dairy = ['Cream', 'Butter', 'Ghee', 'Milk', 'Butter']
Sweet = ['Cane', 'Honey', 'Sugar', 'Sweet', 'Sweeteners', 'Candy', 'Choco']
Infant = ['Infant food']
Other = ['Miscellaneous']
Vegetable = ['Pimento', 'Vegetables', 'Tomatoes', 'Pea', 'Vegetable', 'Cabbage', 'Broccoli', 'Cauliflower', 'Kale', 'Sprout', 'Cruciferous', 'Cucumber']
Palm = ['Palm']
Soy = ['Soy']
Coconut = ['Coconut', 'Copra']
list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']

map_list = [Oil, Grain, Spice, Plant, Fruit, Seafood, Seed_Nut, Alcohol, Dry_Goods, Meat, Dairy, Sweet, Infant, Other, Vegetable, Palm, Soy, Coconut]


cat_zip = list(zip(list_cats, map_list))
cat_zip
map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict
#print(list(zip(Oil)))
#print(list(zip(map_list)))
#cat_map = {[Oil, Grain]}

{'Oil': [['Oilcrops', 'Oil']],
 'Grain': [['Pulse',
   'Cereal',
   'Rice',
   'Barley',
   'Oat',
   'Maize',
   'Bean',
   'Rye',
   'Millet',
   'Sorghum',
   'Wheat',
   'Lentil',
   'Lentil']],
 'Spice': [['Pepper', 'Cloves', 'Spice']],
 'Plant': [['Cassava',
   'Root',
   'Plant',
   'Yam',
   'Plantain',
   'potatoes',
   'potato',
   'Onions']],
 'Fruit': [['Date',
   'Banana',
   'Pineapple',
   'Grape',
   'Lemon',
   'Lime',
   'Fruits',
   'Apples',
   'Olives',
   'Fruit',
   'Grapefruit',
   'Orange',
   'Mandarin',
   'Citrus']],
 'Seafood': [['Cephalopod',
   'Squid',
   'Cuttlefish',
   'Octopus',
   'Aquatic',
   'Crustacean',
   'Crab',
   'Shrimp',
   'Langoustine',
   'Lobster',
   'Sea',
   'Fish',
   'Pelagic',
   'Molluscs',
   'Clam',
   'Oyster',
   'Mussel',
   'Marine',
   'Seafood',
   'Shellfish',
   'Freshwater']],
 'Seed_Nut': [['Seed',
   'Nut',
   'Cottonseed',
   'Rapeseed',
   'Mustardseed',
   'Sunflower',
   'Sesame',
   'Groundnut',
   'Peanut',
 

In [33]:
# length of ingredient list
len(OFD_Ing_List)

37355

In [34]:
# Effort to loop through ingredients list
for k, v in map_dict.items():
    for i in OFD_Ing_List:
        if i in k:
            print(i)
            print(f'--------------------------------------------\n' 
                  f'{i} IS in dict in {k}, {v}\n'
                  f'--------------------------------------------\n')
        #else:
        #    print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n{i} is NOT in dict\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n')


--------------------------------------------
 IS in dict in Oil, [['Oilcrops', 'Oil']]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [['Oilcrops', 'Oil']]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [['Oilcrops', 'Oil']]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [['Oilcrops', 'Oil']]
--------------------------------------------

Oil
--------------------------------------------
Oil IS in dict in Oil, [['Oilcrops', 'Oil']]
--------------------------------------------

Oil
--------------------------------------------
Oil IS in dict in Oil, [['Oilcrops', 'Oil']]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [['Oilcrops', 'Oil']]
--------------------------------------------


--------------------------------------------
 IS 

--------------------------------------------
Coconut IS in dict in Coconut, [['Coconut', 'Copra']]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [['Coconut', 'Copra']]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [['Coconut', 'Copra']]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [['Coconut', 'Copra']]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [['Coconut', 'Copra']]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [['Coconut', 'Copra']]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [['Coc

In [35]:
#define function to comb through the other category list
def food(iterable):
    for k, v in map_dict.items():
        if iterable in v:
            print(f'{iterable} is in a category\n----------------------------')
        else:
            print(f'{iterable} is NOT in a category\n-------------------------')
#food(OFD_Ing_List)           

In [36]:
# drop NaN values, group by count of categories_en where count of products > 1, descending 
category_df = food_df.dropna(axis=0, subset=['categories_en'])
grp_ctgry = category_df.groupby(['categories_en'])
grp_cnt = grp_ctgry.count().sort_values('created_t', ascending=False)
grp_cnt_df = pd.DataFrame(grp_cnt['created_t'])
grp_cnt_df = grp_cnt_df.rename(columns={'created_t': 'count'})
grp_cnt_df = grp_cnt_df.loc[grp_cnt_df['count'] > 1, :]
grp_cnt_df

,count
categories_en,
"Beverages,Non-sugared beverages",2345
"Sugary snacks,Biscuits and cakes,Biscuits",893
Fats,732
"Sugary snacks,Chocolates,Dark chocolates",678
"Sugary snacks,Confectioneries,Candies",587
"Groceries,Sauces,Tomato sauces,Ketchup",561
"Salty snacks,Appetizers,Crackers",534
"Dairies,Milks",504
"Sugary snacks,Biscuits and cakes,Biscuits,Chocolate biscuits",498


In [37]:
# drop NaN values, group b brands where count of products > 300, descending 

brand_df = food_df.dropna(axis=0, subset=['brands'])
grp_brnd = brand_df.groupby(['brands'])
brnd_cnt = grp_brnd.count().sort_values('created_t', ascending=False)
brnd_cnt_df = pd.DataFrame(brnd_cnt['created_t'])
brnd_cnt_df = brnd_cnt_df.rename(columns={'created_t': 'count'})
brnd_cnt_df = brnd_cnt_df.loc[brnd_cnt_df['count'] > 300, :]
brnd_cnt_df

,count
brands,
Carrefour,3670
Auchan,3217
U,2608
Casino,2091
Leader Price,2015
Meijer,1995
Kroger,1662
Cora,1374
Ahold,1370


In [38]:
# drop NaN values, group by country where count of products > 300, descending 

country_df = food_df.dropna(axis=0, subset=['countries_en'])
grp_cntry = country_df.groupby(['countries_en'])
cntry_cnt = grp_cntry.count().sort_values('created_t', ascending=False)
cntry_cnt_df = pd.DataFrame(cntry_cnt['created_t'])
cntry_cnt_df = cntry_cnt_df.rename(columns={'created_t': 'count'})
cntry_cnt_df = cntry_cnt_df.loc[cntry_cnt_df['count'] > 1, :]
cntry_cnt_df

,count
countries_en,
United States,173159
France,123961
Switzerland,14932
Germany,8470
Spain,5529
United Kingdom,5092
Belgium,2936
Australia,2201
Russia,1582


In [39]:
# rename FAO data to prep for merge.
cntry_item_grpby = fao_data_df.rename(columns={"Area":"Country"})#.groupby(['Area', 'Item'])
cntry_item_grpby.head()

,Area Abbreviation,Area Code,Country,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [40]:
# Rename items as countries for merge.
#renamed_fao = cntry_item_grpby.rename(columns={"Area":"Country"})
#renamed_fao.head()

In [41]:
#Rename Items column to Country for merge
renamed_clean_df = clean_df.rename(columns={'countries': 'Country'})
#clean_df.groupby(['countries_en','product_name'])
#renamed_ofd = clean_df.rename(columns={'countries_en': 'Country'})
renamed_clean_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,Country,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
176,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.1600,0.8500,0.0,0.0,12.00,0.000,23.00,7.000,6.0,c,6.0,2.000,100 g,NaN,0.05600,NaN,0.0000,0.000000,0.000000,0.000000,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN
182,1484501528,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15T17:32:08Z,0000001938067,Canada,en:canada,Canada,Chaussons tressés aux pommes,Kirkland Signature,kirkland-signature,1090.0,3.33,0.6470,0.2550,0.0,0.0,10.70,24.700,38.70,2.000,9.0,c,9.0,2.000,150 g / 1 chausson,"en:e202,en:e211,en:e330,en:e509,en:e920",0.00533,0.667,0.0133,0.001600,0.000480,0.000000,Biscuits and cakes,Sugary snacks,1.200 kg,"en:sugary-snacks,en:biscuits-and-cakes,en:past...",Sugary snacks,en:sugary-snacks,"Snacks sucrés,Biscuits et gâteaux,Pâtisseries","Sugary snacks,Biscuits and cakes,Pastries",frais,Frais,NaN,NaN,Brossard Québec,NaN,NaN,NaN,Pâtisserie,Costco,Brossard Québec,brossard-quebec,NaN
183,1488464896,http://world-en.openfoodfacts.org/product/0000...,b7,2017-03-02T14:28:16Z,0000004302544,Canada,en:canada,Canada,Pain Burger Artisan,Kirkland Signature,kirkland-signature,1160.0,10.00,1.5200,0.6000,0.0,0.0,1.11,1.110,53.30,0.333,1.0,b,1.0,2.220,90 g / 1 pain,NaN,0.00000,0.000,0.0222,0.000000,0.006000,0.000000,unknown,unknown,1.008 kg / 12 pain,fr:boulange,fr:Boulange,fr:boulange,Boulange,fr:Boulange,"frais,plastique","Frais,plastique",NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Pain,Costco,"Brossard,Québec","brossard,quebec",NaN
185,1420150193,http://world-en.openfoodfacts.org/product/0000...,teolemon,2015-01-01T22:09:53Z,0000007020254,France,en:france,France,Root Beer,A&W,a-w,215.0,0.00,0.0616,0.0242,0.0,0.0,0.00,13.600,14.20,0.000,18.0,e,3.0,0.000,33 cl,"en:e150,en:e211,en:e999",NaN,NaN,NaN,NaN,NaN,NaN,Sweetened beverages,Beverages,33 cl e,"en:beverages,en:carbonated-drinks,en:sodas,en:...",Beverages,en:beverages,"Boissons,Boissons gazeuses,Sodas,Boissons sucr...","Beverages,Carbonated drinks,Sodas,Sugared beve...","canette,metal","Canette,Métal",http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,France,"Kosher,fr:Contient-des-ogm","Kascher,Contient des OGM","en:kosher,fr:contient-des-ogm",NaN,NaN,États-Unis,etats-unis,NaN
187,1491943307,http://world-en.openfoodfacts.org/product/0000...,b7,2017-04-11T20:41:47Z,0000008237798,Canada,en:canada,Canada,Quiche Lorraine,Kirkland Signature,kirkland-signature,478.0,5.36,0.4990,0.1960,0.0,0.0,6.79,0.714,7.86,2.860,2.0,b,2.0,0.357,280 g / 1/5 de la Quiche,"en:e235,en:e316,en:e407",0.01070,0.000,0.0286,0.000429,0.000643,0.000011,Pizza pies and quiche,Composite foods,1 400 kg,"en:meals,en:pizzas-pies-and-quiches,en:quiches...",Meals,en:meals,Quiches lorraines,"Meals,Pizzas pies and quiches,Quiches,fr:Quich...",frai,Frai,NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Quiche lorraine,Costco

In [80]:
# Merge the two dataframes on country.
cntry_merge = pd.merge(renamed_clean_df, cntry_item_grpby, on='Country', how='outer')
cntry_merge.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,Country,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens,Area Abbreviation,Area Code,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.0,2.16,0.85,0.0,0.0,12.0,0.0,23.0,7.0,6.0,c,6.0,2.0,100 g,NaN,0.056,NaN,0.0,0.0,0.0,0.0,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN,CAN,33.0,2511.0,Wheat and products,5521.0,Feed,1000 tonnes,56.13,-106.35,1195.0,1208.0,1462.0,1262.0,1356.0,1437.0,1789.0,1561.0,2640.0,1887.0,2048.0,2087.0,1907.0,1695.0,1790.0,2037.0,2122.0,2302.0,2406.0,2175.0,2001.0,1815.0,2246.0,1979.0,2007.0,3040.0,3796.0,2820.0,2387.0,2552.0,3275.0,3944.0,5584.0,3977.0,3974.0,4398.0,3588.0,4161.0,4268.0,3447.0,3342.0,4066.0,3442.0,5095.0,4152.0,4150.0,2199.0,2197.0,2629.0,1725.0,3338.0,3410.0,3731.0
1,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.0,2.16,0.85,0.0,0.0,12.0,0.0,23.0,7.0,6.0,c,6.0,2.0,100 g,NaN,0.056,NaN,0.0,0.0,0.0,0.0,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN,CAN,33.0,2511.0,Wheat and products,5142.0,Food,1000 tonnes,56.13,-106.35,1447.0,1493.0,1645.0,1406.0,1709.0,1545.0,1575.0,1613.0,1705.0,1713.0,1604.0,1695.0,1738.0,1732.0,1809.0,1933.0,1837.0,1769.0,1785.0,1870.0,1625.0,1897.0,1992.0,1890.0,2099.0,2011.0,1990.0,2148.0,2125.0,2159.0,2076.0,2148.0,2164.0,2505.0,2455.0,2631.0,2665.0,2804.0,2746.0,2724.0,2742.0,2728.0,2740.0,2815.0,2827.0,2854.0,2908.0,2897.0,2895.0,2889.0,2724.0,2992.0,2989.0
2,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.0,2.16,0.85,0.0,0.0,12.0,0.0,23.0,7.0,6.0,c,6.0,2.0,100 g,NaN,0.056,NaN,0.0,0.0,0.0,0.0,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN,CAN,33.0,2805.0,Rice (Milled Equivalent),5142.0,Food,1000 tonnes,56.13,-106.35,31.0,32.0,38.0,37.0,58.0,37.0,40.0,44.0,47.0,51.0,70.0,68.0,67.0,5

In [82]:
# Sort only relevant data columns and remove nana elements.
ord_cntry_merge = merge_by_cntry[['Country','product_name','brands','main_category','Element','brands','manufacturing_places']]
ord_cntry_merge = ord_cntry_merge.rename(columns={'product_name':'Product Name', "main_category":"Main Category", "brands":"Brands",'manufacturing_places':'Manufacturing Places'})
ord_cntry_merge.iloc[0:4900000].fillna('', inplace = True)
ord_cntry_merge.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Country,Product Name,Brands,Main Category,Element,Brands,Manufacturing Places
0,Afghanistan,,,,Food,,
1,Afghanistan,,,,Food,,
2,Afghanistan,,,,Feed,,
3,Afghanistan,,,,Food,,
4,Afghanistan,,,,Feed,,


In [71]:
#ord_cntry_merge.dtypes
#ord_cntry_merge['Item']=ord_cntry_merge['Item'].astype(str)+'_'+ord_cntry_merge['main_category']
#ord_cntry_merge["period"] = ord_cntry_merge["Item"].map(str) + ord_cntry_merge["main_category"].map(str)
#ord_cntry_merge.head()
#ord_cntry_merge['Items'] = ord_cntry_merge['Item'] + ord_cntry_merge['product_name']
#ord_cntry_merge.head()
#ord_cntry_merged = ord_cntry_merge["Item"].map(str) + ord_cntry_merge["main_category"].map(str)
#ord_cntry_merge['Item', 'product_name'].str.cat(sep='')
#ord_cntry_merge.iloc[0:50] = ord_cntry_merge["Item"]+ "-" + ord_cntry_merge["product_name"].map(str) 
#ord_cntry_merge.head()
ord_cntry_merge = ord_cntry_merge.copy()
#ord_cntry_merge.iloc[0:50] = ord_cntry_merge["Item"]+ "-" + ord_cntry_merge["product_name"].map(str) 
#ord_cntry_merge.head()

AttributeError: 'function' object has no attribute 'copy'

In [54]:
# Use table summaries to decide what cleaning needs to be done. 
print(len(ord_cntry_merge))


4913848
